In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers
import config


from classification.models.M5 import M5, M5PLModule

from utils.RobustnessExperiment import load_module
from utils.Visual import show_audio, show_spectogram, show_waveplot

/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [4]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Train Model

In [10]:
from classification.models.M5 import M5, M5PLModule
from attacks.pgd import ProjectedGradientDescent


hparams = {
    "batch_size": 64,
    "learning_rate": 3e-4,
    "weight_decay": 0.001,
    "lr_decay": 0.95
}

attack_args = {"norm":"inf", "epsilon": 0.15}
current_attack = ProjectedGradientDescent

model = M5PLModule(hparams)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')
model.set_smooth(0.5)
#model.setAttack(current_attack, attack_args)

In [11]:
trainer = pl.Trainer(
    max_epochs=20,
    #logger= loggers.TensorBoardLogger(config.LOG_DIR, name="M5"),
    gpus=1 if torch.cuda.is_available() else None,
    log_gpu_memory='all'
)


trainer.fit(model)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
Set SLURM handle signals.

   | Name           | Type         | Params
--------------------------------------------
0  | model          | M5           | 555 K 
1  | model.model    | Sequential   | 555 K 
2  | model.model.0  | Conv1d       | 10 K  
3  | model.model.1  | BatchNorm1d  | 256   
4  | model.model.2  | MaxPool1d    | 0     
5  | model.model.3  | Dropout      | 0     
6  | model.model.4  | Conv1d       | 49 K  
7  | model.model.5  | BatchNorm1d  | 256   
8  | model.model.6  | MaxPool1d    | 0     
9  | model.model.7  | Dropout      | 0     
10 | mod

Val-Acc=0.03793716656787196


/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Val-Acc=0.7777119146413752


Val-Acc=0.6941315945465323


...

Val-Acc=0.6277415530527564


Val-Acc=0.7196206283343213


Val-Acc=0.6520450503852994


Val-Acc=0.6674570243034973



1

In [12]:
model.save("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/vanilla_smoothed20.p")

Saved model to "/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/vanilla_smoothed20.p"


In [ ]:
####model.save("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/vanilla_model.p")

#model = load_module("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_attack_ProjectedGradientDescent_v77.p", M5PLModule)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')
model.cuda()
model.report()

### Run a single attack

In [ ]:
from attacks.pgd import ProjectedGradientDescent
attack = ProjectedGradientDescent(model.model, model.val_dataloader(), {"norm":"inf", "epsilon": 0.1, "num_iter":1}, early_stopping=100, device='cuda', save_samples=True)
attack.attack()
attack.report( )
attack.showAdversarialExample(target_class=1)

# L2: alle positiv: ab 140. -> bis 150
# LInf: ab 0.5
# L1: ab 30K -> bis 50K

In [ ]:
attack.showAdversarialExample(target_class=1)

In [ ]:
from attacks.volume import VolumeAttack
attack = VolumeAttack(model.model, model.val_dataloader(), {"epsilon":0.9}, early_stopping=-1, device='cuda', save_samples=True)

attack.attack()
attack.report( )

In [ ]:
attack.showAdversarialExample(target_class=1)

In [ ]:
from attacks.pitch import PitchAttack
attack = PitchAttack(model.model, model.val_dataloader(), {}, early_stopping=-1, device='cuda', save_samples=True)

attack.attack()
attack.report( )

In [ ]:
attack.showAdversarialExample(target_class=1)

In [ ]:
from attacks.speed import TimeStretchAttack
attack = TimeStretchAttack(model.model, model.val_dataloader(), {}, early_stopping=-1, device='cuda', save_samples=True)

attack.attack()
attack.report( )

In [ ]:
attack.showAdversarialExample(target_class=1)

* change:
    * return accuracy
    * make easy to compare via spectogram & waveform


In [ ]:
##############################################################################################################
##############################################################################################################
##############################################################################################################

from utils.RobustnessExperiment import RobustnessExperiment, load_experiment
from attacks.FGA_Batch import fast_gradient_attack
from attacks.pgd import ProjectedGradientDescent
from classification.models.M5 import M5PLModule

exp_config = [{
              "attack_fn": VolumeAttack, 
              "attack_arg": {"epsilon": [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]},
              "meta": {"key_result":"success_rate", "key_config":"epsilon", "title":"Volume"}
             }]

# uncomment this if you want to create a new experiment instead!
#experiment = RobustnessExperiment(exp_config, title="VOLUME_ONLY")
experiment = load_experiment(exp_folder = "0005_VOLUME_ONLY")

#experiment.run("./1592000682.p", M5PLModule) #20 epochs normal training with hparams above
experiment.run("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_attack_ProjectedGradientDescent_v8_epoch_25.p", M5PLModule) #5 epochs adv training with hparams above

models = ['/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_attack_ProjectedGradientDescent_v69_epoch_10.p',
          '/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_attack_ProjectedGradientDescent_v69.p',]

for m in models:
    experiment.run(m, M5PLModule)
##############################################################################################################
##############################################################################################################
##############################################################################################################

In [ ]:
from attacks.pgd import ProjectedGradientDescent
from utils.RobustnessExperiment import load_experiment
experiment = load_experiment(exp_folder = "0007_QUICK_EVALUATION")#"0001_EVALUATION_STANDARD")

In [ ]:
experiment.show_evaluated_models()

In [ ]:
experiment.show_best_models(metric = "success_rate", best_n = 1, limit_eps=100)


def get_model_performance(model_name):
    total_sum = 0
    for i, attack in enumerate(experiment.all_results.keys()):
        #print(experiment.all_results[attack][model_name])
        total_sum += sum([ res["acc"] for res in experiment.all_results[attack][model_name]][:6] )
    return total_sum


best = []
for m in experiment.show_evaluated_models():
    try:
        best.append((get_model_performance(m), m))
    except:
        print("jo")
    
print(sorted(best))

experiment.get_model_performance("run1_v3.p")

In [ ]:
experiment.compare(config_key = "epsilon", results_key = "success_rate", models=['M5_attack_ProjectedGradientDescent_v8_epoch_25.p',
 'run1_v3.p'])